# Intermediate Loop Setup and Optimization

Testing intermediate loop setup with scipy optimize

## Method 1: Optimization of loop temperatures
Uses scipy.optimize to determine loop temperatures for intermediate and secondary loop for SC-CO2 cycle.

In [71]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.optimize as spo
import read_inputs, compute_cycle_efficiency
from compute_required_area_v2 import _compute_velocity
import compute_required_area_v2 as compute_required_area

from th_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Method 1.1: Optimization of hot leg temperature and loop dT
4 input variables are used:
1. Intermediate Loop Hot Leg Temperature
2. Intermediate Loop (Hot-Cold) Leg dT
3. Secondary Loop Hot Leg Temperature
4. Secondary Loop (Hot-Cold) Leg dT


In [69]:
""" Assume x to have the following structure
         0    1       2    3
x: [T2_hot, dT2, T3_hot, dT3]
"""
inputs = read_inputs.read_inputs()
primary_hot = inputs["Primary Hot Temperature (C)"]
primary_cold = inputs["Primary Cold Temperature (C)"]
primary_dT = primary_hot - primary_cold
# yapf:disable
cons = ({'type': 'ineq', 'fun': lambda x: x[3] - x[1]},
        {'type': 'ineq', 'fun': lambda x: primary_cold - (x[0]-x[1])}, # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: (x[0]-x[1]) - (x[2]-x[3])},  # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: x[0] - x[2]},                # Hot Leg T gap
        {'type': 'ineq', 'fun': lambda x: 350000 -hx_calc(x, return_cost=True)}
       )
bounds = spo.Bounds(
    #         T2_hot,   dT2,       T3_hot,   dT3  Cost
    lb=[primary_cold, 150.0, primary_cold, 150.0], # Lower Bound
    ub=[ primary_hot, 200.0,  primary_hot, 200.0], # Upper Bound
    keep_feasible=True
)

# yapf:enable
def hx_calc(x, print_res=False, return_cost=False):
    t2h, dT2, t3h, dT3 = x
    t2c = t2h - dT2
    t3c = t3h - dT3
    inputs = read_inputs.read_inputs()
    inputs['Intermediate Hot Temperature (C)'] = t2h
    inputs['Intermediate Cold Temperature (C)'] = t2c
    inputs['Secondary Hot Temperature (C)'] = t3h
    inputs['Secondary Cold Temperature (C)'] = t3c
    try:
        area_results = compute_required_area.compute_required_area(inputs)
        channel_thickness = 3*inputs["Plate thickness (m)"] + inputs[
            "Channel Diameter (m)"]
        channel_width = inputs["Plate thickness (m)"] + inputs[
            "Channel Diameter (m)"]
        channel_volume = channel_thickness*channel_width*area_results[
            "Heat Exchanger Length (m)"]
        HX_volume = sum(channel_volume*area_results["Number of Channels"])
        inflation = 1.62
        cost_conversion = 132000                          # $/m3
        cost = HX_volume*132000*inflation                 # $
        LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(
            np.log((primary_hot-t3h)/(primary_cold-t3c)))
    except:
        return 1e5
    if print_res:
        print(f"Cost Fns: {LMTD}, {(5e-3*cost**(2/3))}")
        print(f"Temperatures: {t2h}, {t2c}, {t3h}, {t3c}")
        print(f"dTs         : {t2h-t2c}, {t3h-t3c}")
        print("Vol:  ", channel_volume*area_results["Number of Channels"])
        print('Cost: ', cost)
        print(pd.Series(inputs))
        print(pd.Series(area_results))
        print(
            pd.Series(
                compute_cycle_efficiency.compute_cycle_efficiency(inputs)))
    if return_cost:
        return cost
    return LMTD + 5e-3*cost**(2/3)

# res = spo.minimize(hx_calc, [530, 530 - 155, 500, 500 - 180], bounds=bounds, constraints=cons)

init_guess = [primary_hot - 20, 165, primary_hot - 50, 190]
res = spo.minimize(hx_calc, init_guess, bounds=bounds, constraints=cons)
hx_calc(res.x, True)

/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LMTD2 = (deltaT_2h-deltaT_2c)/np.log(deltaT_2h/deltaT_2c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LMTD2 = (deltaT_2h-deltaT_2c)/np.log(deltaT_2h/deltaT_2c)
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LMTD2 = (deltaT_2h-deltaT_2c)/np.log(deltaT_2h/deltaT_2c)
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LM

Cost Fns: 78.17832869358521, 24.61927438158612
Temperatures: 527.1421160578259, 365.8503957698766, 477.33166831191835, 316.0399480239692
dTs         : 161.29172028794926, 161.29172028794915
Vol:   [0.58274667 1.03298382]
Cost:  345507.80805387575
Thermal Power (MW)                              30
Primary Fluid                               Sodium
Primary Hot Temperature (C)                    550
Primary Cold Temperature (C)                   400
Primary Mass Flow Rate (kg/s)                  NaN
Primary Pressure (kPa)                         300
Secondary Fluid                      CarbonDioxide
Secondary Hot Temperature (C)           477.331668
Secondary Cold Temperature (C)          316.039948
Secondary Mass Flow Rate (kg/s)                NaN
Secondary Pressure (kPa)                     25000
Plate thickness (m)                          0.001
Channel Diameter (m)                       0.00135
Plate material                               SS316
Primary Pressure Drop (kPa)            

102.79760307517134

### Method 1.2: Optimization of hot and cold leg temperature 
4 input variables are used:
1. Intermediate Loop Hot Leg Temperature
2. Intermediate Loop Cold Leg Temperature
3. Secondary Loop Hot Leg Temperature
4. Secondary Loop Cold Leg Temperature


In [70]:
""" Assume x to have the following structure
         0        1       2        3   4   5
x: [T2_hot, T2_cold, T3_hot, T3_cold, L1, L2]
"""
inputs = read_inputs.read_inputs()
primary_hot = inputs["Primary Hot Temperature (C)"]
primary_cold = inputs["Primary Cold Temperature (C)"]

#yapf:disable
cons = ({'type': 'ineq', 'fun': lambda x: (x[0] - x[1]) - (primary_hot-primary_cold)}, # dT1 > dT2
        {'type': 'ineq', 'fun': lambda x: (x[2] - x[3]) - (x[0] - x[1])},                # dT2 > dT3
        {'type': 'ineq', 'fun': lambda x: primary_cold - x[1]}, # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: x[1] - x[3]},         # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: primary_hot - x[0]},  # Hot Leg T gap
        {'type': 'ineq', 'fun': lambda x: x[0] - x[2]},         # Hot Leg T gap
        {'type': 'ineq', 'fun': lambda x: 350000 -hx_calc(x, return_cost=True)}
       )
bounds = spo.Bounds(
    #         T2_hot,      T2_cold,        T3_hot,     T3_cold  Ch.Diam  dP1  dP2
    lb=[primary_cold,          200, primary_cold,          200], # Lower Bound
    ub=[ primary_hot, primary_cold,  primary_hot, primary_cold], # Upper Bound
)

# yapf:enable
def hx_calc(x, print_res=False, return_cost=False):
    t2h, t2c, t3h, t3c = x
    inputs = read_inputs.read_inputs()
    inputs['Intermediate Hot Temperature (C)'] = t2h
    inputs['Intermediate Cold Temperature (C)'] = t2c
    inputs['Secondary Hot Temperature (C)'] = t3h
    inputs['Secondary Cold Temperature (C)'] = t3c
    try:
        area_results = compute_required_area.compute_required_area(inputs)
        channel_thickness = 3*inputs["Plate thickness (m)"] + inputs[
            "Channel Diameter (m)"]
        channel_width = inputs["Plate thickness (m)"] + inputs[
            "Channel Diameter (m)"]
        channel_volume = channel_thickness*channel_width*area_results[
            "Heat Exchanger Length (m)"]
        HX_volume = sum(channel_volume*area_results["Number of Channels"])
        inflation = 1.62
        cost_conversion = 132000                          # $/m3
        cost = HX_volume*132000*inflation                 # $
        LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(
            np.log((primary_hot-t3h)/(primary_cold-t3c)))
    except:
        return 1e7
    if print_res:
        print(f"Cost Fns: {LMTD}, {(cost**(2/3)*5e-3)}")
        print(f"Temperatures: {t2h}, {t2c}, {t3h}, {t3c}")
        print(f"dTs         : {t2h-t2c}, {t3h-t3c}")
        print("Vol:  ", channel_volume*area_results["Number of Channels"])
        print('Cost: ', cost)
        print(pd.Series(inputs))
        print(pd.Series(area_results))
        print(
            pd.Series(
                compute_cycle_efficiency.compute_cycle_efficiency(inputs)))
    if return_cost:
        return cost
    return LMTD + 5e-3*cost**(2/3)

res = spo.minimize(
    hx_calc, [530, 530 - 165, 500, 500 - 190], bounds=bounds, constraints=cons)
# res = spo.minimize(hx_calc, [545, 545 - 155, 525, 525 - 180], bounds=bounds, constraints=cons)
hx_calc(res.x, True)

/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/tmp/ipykernel_54540/283989095.py:41: RuntimeWarning: invalid value encountered in scalar divide
  LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(np.log((primary_hot-t3h)/(primary_cold-t3c)))
/tmp/ipykernel_54540/283989095.py:41: RuntimeWarning: invalid value encountered in scalar divide
  LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(np.log((primary_hot-t3h)/(primary_cold-t3c)))
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid val

Cost Fns: 76.80000000000008, 24.855995301447408
Temperatures: 524.0269052604001, 374.02690526040004, 472.45205167905635, 322.4520516790565
dTs         : 150.00000000000006, 149.99999999999983
Vol:   [0.62414886 1.01494108]
Cost:  350502.99291649996
Thermal Power (MW)                              30
Primary Fluid                               Sodium
Primary Hot Temperature (C)                    550
Primary Cold Temperature (C)                   400
Primary Mass Flow Rate (kg/s)                  NaN
Primary Pressure (kPa)                         300
Secondary Fluid                      CarbonDioxide
Secondary Hot Temperature (C)           472.452052
Secondary Cold Temperature (C)          322.452052
Secondary Mass Flow Rate (kg/s)                NaN
Secondary Pressure (kPa)                     25000
Plate thickness (m)                          0.001
Channel Diameter (m)                       0.00135
Plate material                               SS316
Primary Pressure Drop (kPa)          

101.65599530144749

## Method 2: Optimization of loop parameters
Uses scipy.optimize to determine optimal HX parameters given fixed loop temperatures (primary, intermediate, secondary)

In [18]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.optimize as spo
import sys
from th_functions import *
import warnings

sys.path.append('../')
from compute_required_area_v2 import compute_required_area
from compute_cycle_efficiency import compute_cycle_efficiency
from loop_sizing import gen_input

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
def size_loop(inputs):
    def _hx_calc(x, return_cost=False, return_results=False):
        t2h, t2c, t3h, t3c, dp1, dp2 = x
        inputs['Intermediate Hot Temperature (C)'] = t2h
        inputs['Intermediate Cold Temperature (C)'] = t2c
        inputs['Secondary Hot Temperature (C)'] = t3h
        inputs['Secondary Cold Temperature (C)'] = t3c
        inputs["Primary Pressure Drop (kPa)"] = dp1
        inputs["Secondary Pressure Drop (kPa)"] = dp2
        try:
            area_results = compute_required_area(inputs)
            plate_thicknesses = np.array([
                inputs["HX1 Plate thickness (m)"],
                inputs["HX2 Plate thickness (m)"],
            ])
            plate_lengths = np.array([
                area_results["HX1 Length (m)"],
                area_results["HX2 Length (m)"],
            ])
            hx_num_channels = np.array([
                area_results["HX1 Number of Channels"],
                area_results["HX2 Number of Channels"]
            ])

            channel_thickness = inputs[
                "HX Channel Diameter (m)"] + 3*plate_thicknesses
            channel_width = inputs["HX Channel Diameter (m)"] + plate_thicknesses
            channel_volumes = channel_thickness*channel_width*plate_lengths

            HX_volumes = channel_volumes*hx_num_channels

            inflation = 1.62
            cost_conversion = 132000                                  # $/m3
            cost = sum(HX_volumes)*132000*inflation                   # $
            LMTD = (((primary_hot-t3h) - (primary_cold-t3c))/(np.log(
                (primary_hot-t3h)/(primary_cold-t3c))))
        except:
            return 1e7
        if return_cost:
            return cost
        if return_results:
            loop = {}
            loop = {**loop, **area_results}
            loop["HX Cost ($)"] = cost
            loop = {**inputs, **loop, **(compute_cycle_efficiency(inputs))}
            return loop
        return LMTD + 5e-3*cost**(2/3)

    def _pipe_calc(loop_res):
        # Determines the size of piping and the frictional pressure drop
        # within the pipes for the primary loop
        loop_idx_dict = {0: "Primary", 1: "Intermediate", 2: "Secondary"}
        loop_idx_mat = {0: "Sodium", 1: "Sodium", 2: "CarbonDioxide"}
        loop_idx_length = {0: 15, 1: 15, 2: 15}

        dP = []
        for i in range(3):
            loop = loop_idx_dict[i]
            mdot = loop_res[f"{loop} Mass Flow Rate (kg/s)"]
            tavg = (loop_res[f"{loop} Hot Temperature (C)"]
                    + loop_res[f"{loop} Cold Temperature (C)"])/2 + 273
            P = loop_res[f"{loop} Pressure (kPa)"]*1e3
            cp, k, rho, mu = fluid_properties(loop_res[f"{loop} Fluid"], tavg, P)

            pipe_radius = 0.587/2*np.sqrt(mdot/1256)
            velocity = mdot/rho/(np.pi*pipe_radius**2)
            dP.append(
                compute_pressure_drop(velocity,
                                      rho,
                                      2*pipe_radius,
                                      mu,
                                      loop_idx_length[i]))
            loop_res[f"{loop_idx_dict[i]} Pipe Pressure Drop (Pa)"] = dP[i]
        return loop_res

    primary_hot = inputs["Primary Hot Temperature (C)"]
    primary_cold = inputs["Primary Cold Temperature (C)"]
    primary_dT = primary_hot - primary_cold

    # yapf:disable
    cons = ({'type': 'ineq', 'fun': lambda x: (x[0] - x[1]) - primary_dT},    # dT1 > dT2
            {'type': 'ineq', 'fun': lambda x: (x[2] - x[3]) - (x[0] - x[1])}, # dT2 > dT3
            {'type': 'ineq', 'fun': lambda x: primary_hot - x[0]},            # Hot Leg T gap
            {'type': 'ineq', 'fun': lambda x: x[0] - x[2]},                   # Hot Leg T gap
            {'type': 'ineq', 'fun': lambda x: primary_cold - x[1]},           # Cold Leg T gap
            {'type': 'ineq', 'fun': lambda x: x[1] - x[3]},                   # Cold Leg T gap
            {'type': 'ineq', 'fun': lambda x: 450000 -_hx_calc(x, return_cost=True)}
           )
    bounds = spo.Bounds(
        #         T2_hot,      T2_cold,        T3_hot,     T3_cold   dP1  dP2
        lb=[primary_cold,          100, primary_cold,           50,  30,  30], # Lower Bound
        ub=[ primary_hot, primary_cold,  primary_hot, primary_cold, 200, 200], # Upper Bound
    )
    # yapf:enable
    init_guess = [
        primary_hot - 15,
        primary_hot - primary_dT - 25,
        primary_hot - 35,
        primary_hot - primary_dT - 55,
        100,
        100,
    ]
    warnings.simplefilter(action='ignore', category=RuntimeWarning)
    opti_res = spo.minimize(
        _hx_calc, init_guess, bounds=bounds, constraints=cons)
    loop_res = _hx_calc(opti_res.x, return_cost=False, return_results=True)
    loop_res = _pipe_calc(loop_res)
    rem_list = [
        "Thermal Power (MW)",
        "Step size (m)",
        "HX length lower bound (m)",
        "HX length upper bound (m)"
    ]
    for key in rem_list:
        del loop_res[key]
    return loop_res

In [44]:
input = gen_input()
size_loop(input)

{'Primary Fluid': 'Sodium',
 'Primary Hot Temperature (C)': 550,
 'Primary Cold Temperature (C)': 400,
 'Primary Mass Flow Rate (kg/s)': 157.0904027686638,
 'Primary Pressure (kPa)': 300,
 'Intermediate Fluid': 'Sodium',
 'Intermediate Hot Temperature (C)': 520.2612018014496,
 'Intermediate Cold Temperature (C)': 370.2611974384153,
 'Intermediate Mass Flow Rate (kg/s)': 156.37928069706126,
 'Intermediate Pressure (kPa)': 300,
 'Secondary Fluid': 'CarbonDioxide',
 'Secondary Hot Temperature (C)': 451.135361103637,
 'Secondary Cold Temperature (C)': 301.12863251534196,
 'Secondary Mass Flow Rate (kg/s)': 159.71264503615632,
 'Secondary Pressure (kPa)': 25000,
 'HX1 Plate thickness (m)': 0.001,
 'HX1 Plate material': 'SS316',
 'HX2 Plate thickness (m)': 0.002,
 'HX2 Plate material': 'SS316',
 'HX Channel Diameter (m)': 0.0025,
 'Primary Pressure Drop (kPa)': 182.8286703904458,
 'Secondary Pressure Drop (kPa)': 199.99999999999972,
 'Pump/Compressor Efficiency': 0.88,
 'Turbine Efficiency':